In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

## 开始优化
现在有数据：<br />
1. 沪深300指数数据：df300
2. 沪深300指数近3年每天的数据：df300_daily
3. 成分股近3年每天的数据：df
4. 成分股近3年涨跌幅时序数据：df1
5. 指数权重前20支股票的代码与权重值：df20
<br />

目标是最小化绝对误差，找到一组不超过20支股票买入的权重w

## 导入数据

In [2]:
# 指数权重前20的成分股近3年每天的数据
df=pd.read_excel("df_all_top_20.xlsx",index_col=0)
# 指数权重前20的成分股近3年涨跌幅时序数据
df1=pd.pivot_table(df,index=["trade_date","ts_code"],values=["pct_chg"])
# 沪深300指数基本数据
df300=pd.read_excel("df_300_w.xlsx",index_col=0)
df20=df300.loc[:19,:] # 指数权重前20的成分股基本数据
# 沪深300指数近3年每天的数据
df300_daily=pd.read_excel('沪深300指数近3年.xlsx')

H:\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
ts_code_20=df20['ts_code'].tolist()
chg300=df300_daily[['日期Date','涨跌幅(%)Change(%)']]
index2codes=ts_code_20
chgi=df1

## 定义目标函数

In [4]:
def objective(x):
    # 最小化涨跌幅绝对误差之和
    days=len(chg300)
    abs_error=0
    for i in range(days):
        # 计算20支股票组合的当日涨跌幅
        chg_i=0
        date=chg300.loc[i,'日期Date']
        str_query='trade_date == ['+ str(date)+']'
        df_i=chgi.query(str_query)
        df_i=df_i.reset_index()
        for j in range(len(x)):
            # 根据索引找到股票代码，然后找到当日chg
            item=df_i[df_i['ts_code']==index2codes[j]]
            if not item.empty:
                chg_i_j=np.sum(item['pct_chg']/100)
            else:
                # 这只股票当天没数据的话就等于0
                chg_i_j=0
            chg_i+=chg_i_j*x[j]
        # 找到当日沪深300的涨跌幅
        chg300_i=chg300.loc[i,'涨跌幅(%)Change(%)']/100
        # 当日涨跌幅绝对误差(单位：1)
        abs_error_i=np.abs(chg300_i-chg_i)
        # 求和
        abs_error+=abs_error_i
    return abs_error

## 定义约束条件

In [5]:
# 定义约束条件
def constraint1(x):
    return np.sum(x)-1        # 等式约束  

## 给出变量取值范围

In [6]:
nums=20
bounds=[]
for _ in range(nums):
    bounds.append((0.0,1))
bounds=tuple(bounds)

## 开始优化

In [7]:
con1 = {'type': 'eq', 'fun': constraint1}
cons = ([con1])  

x0=np.array([0.05]*20) #定义初始值
solution = minimize(objective, x0, method='SLSQP', \
                bounds=bounds, constraints=cons)

x = solution.x

print('目标值: ' + str(objective(x)))
print('最优解为')
print(x)

目标值: 2.0532077849552643
最优解为
[1.26319738e-01 1.11021969e-01 6.07959963e-02 4.60272301e-02
 5.83096994e-02 3.30104101e-02 2.77288026e-02 6.30228247e-02
 3.30104101e-02 3.46738804e-02 7.45570318e-02 3.30104101e-02
 6.58007534e-02 3.56000395e-18 9.29214484e-03 3.30104101e-02
 3.30104101e-02 6.25826060e-02 6.18048631e-02 3.30104101e-02]


In [8]:
solution

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 2.0532077849552643
       x: [ 1.263e-01  1.110e-01 ...  6.180e-02  3.301e-02]
     nit: 30
     jac: [-1.842e-02 -2.055e-03 ...  9.256e-03  0.000e+00]
    nfev: 665
    njev: 30

## 评价指标

In [14]:
def get_total_return(x):
    # 计算收益率
    days=len(close300)
    total_return=0
    daily_returns=[]
    track_error=0
    earning_rate=[]
    for i in range(days):
        if i==0:
            continue
        # 计算20支股票组合的当日收益率：(当日收盘价 - 前一日收盘价) / 前一日收盘价
        earning_rate_i=0
        date=close300.loc[i,'日期Date']
        str_query='trade_date == ['+ str(date)+']'
        df_i=closei.query(str_query)
        df_i=df_i.reset_index()
        for j in range(len(x)):
            # 根据索引找到股票代码，然后计算当日收益率
            item=df_i[df_i['ts_code']==index2codes[j]]
            if not item.empty:
                earning_rate_i_j=np.sum((item['close']-item['pre_close'])/item['pre_close'])
            else:
                # 这只股票当天没数据的话就等于0
                earning_rate_i_j=0
            earning_rate_i+=earning_rate_i_j*x[j]

        # 找到当日沪深300的收益率
        earning_rate300_i=(close300.loc[i,'收盘Close']-close300.loc[i-1,'收盘Close'])/close300.loc[i-1,'收盘Close']
        # 当日跟踪误差误差(单位：1)
        earning_rate.append(earning_rate_i-earning_rate300_i)
        total_return+=earning_rate_i
        daily_returns.append(earning_rate_i)
    daily_returns=np.array(daily_returns)
    # 求标准差
    track_error=np.std(earning_rate)
    return total_return,daily_returns,track_error

In [21]:
eva_index={}
ts_code_20=df20['ts_code'].tolist()
close300=df300_daily[['日期Date','收盘Close']]
index2codes=ts_code_20
df2=pd.pivot_table(df,index=["trade_date","ts_code"],values=["close","pre_close"])
closei=df2
# 该组合跟踪误差
total_return,daily_returns,track_error=get_total_return(x)
eva_index['跟踪误差']=track_error

# 年化收益率
n=3 # 取的是近3年的数据
annualized_return = (1 + total_return) ** (1 / n) - 1
eva_index['年化收益率']=annualized_return

# 年化波动率
# 计算日波动率（日收益率的标准差）
daily_volatility=np.std(daily_returns, ddof=1)
# 将日波动率年化
annualized_volatility=daily_volatility * np.sqrt(252)
eva_index['年化波动率']=annualized_volatility

# 夏普比率=（策略年化收益率 - 无风险年化收益率 ） / 策略年化波动率
R_p = annualized_return  # 投资组合年化回报率
R_f = 0.0125  # 无风险利率：3年期国债收益率
sigma_p = annualized_volatility  # 投资组合的年化波动率
sharpe_ratio = (R_p - R_f) / sigma_p
eva_index['夏普比率']=sharpe_ratio

In [22]:
eva_index

{'跟踪误差': 0.0037896002541941928,
 '年化收益率': 0.054033011105440565,
 '年化波动率': 0.17371873528856108,
 '夏普比率': 0.2390819334279105}